# import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
%matplotlib inline
import matplotlib as mpl
import math
from dwave.system import DWaveSampler
import minorminer as mm
import dwave.inspector

In [2]:
f = open("token.txt", "r")
sampler = DWaveSampler(solver=dict(topology__type='pegasus'),token=f.read())
print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system6.1 was selected.


# target graph (pegasus)

In [3]:
#create a target graph
T=nx.Graph()
T.add_edges_from(sampler.edgelist)
T.add_nodes_from(sampler.nodelist)

# one long chain

In [58]:
#create a source graph
S=nx.Graph()
#spin chain length
N=3500
#define the nodes
S.add_nodes_from(list(range(N)))
spins=list(S.nodes)
#define the edges. the first node is also the last node (PBC)
for i in range(len(spins)-1):
    S.add_edge(spins[i],spins[i+1])
#impose PBC
S.add_edge(spins[len(spins)-1],spins[0])

In [59]:
cond=0
count=0

while (cond==0 and count<1000):
    print(count)

    embedding=mm.find_embedding(S,T)

    #here we make sure that minorminor didn't make any chains 
    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    #if the embedding has any chains we reject it and try again
    if len(biggerthan1)>0:
        print(len(biggerthan1),"chains")
        cond=0
        count+=1
    if count==999:
        print("reached max iterations")   
    
    #if there are no chains it's a goood embedding
    if len(biggerthan1)==0:
        cond=1
    
print(embedding)

0
6 chains
1


KeyboardInterrupt: embedding cancelled by keyboard interrupt

In [32]:
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})

# many small chains

In [22]:
#create a source graph
S=nx.Graph()
#spin chain length
Nlen=12
#number of chains
Nchains=200

#define the nodes in base chain
base_nodes=[]
for i in range(Nlen):
    base_nodes.append(i)
base_nodes=np.array(base_nodes)
#generate all the nodes
allnodes=[]
for i in range(Nchains):
    allnodes.append(list(base_nodes+Nlen*i))
#all_nodes is a list of length Nchains, each element is a list of length Nlen

#add nodes to source graph
for i in range(len(allnodes)):
    for j in range(len(allnodes[i])):
        S.add_node(allnodes[i][j])

#define the edges
#loop over all the chains you are trying to embed
for i in range(len(allnodes)):
    #extract current list of sites for this ring
    spinlist=allnodes[i]
    for j in range(len(spinlist)-1):
        #add the nearest neighbor interaction as an edge
        S.add_edge(spinlist[j],spinlist[j+1])
    #add the final edge which completes the periodic chain
    S.add_edge(spinlist[len(spinlist)-1],spinlist[0])

In [23]:
#use minorminer to embed this graph (it is really Nchain disconnect graphs)

cond=0
count=0

while (cond==0 and count<1000):
    print(count)

    embedding=mm.find_embedding(S,T)

    #here we make sure that minorminor didn't make any chains 
    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    #if the embedding has any chains we reject it and try again
    if len(biggerthan1)>0:
        print(len(biggerthan1),"chains")
        cond=0
        count+=1
    if count==999:
        print("reached max iterations")   
    
    #if there are no chains it's a goood embedding
    if len(biggerthan1)==0:
        cond=1
    
print(embedding)

density = len(embedding)/sampler.properties["num_qubits"]

print("density of qubits used: {}".format(density))

0
{0: [4426], 1: [233], 11: [368], 2: [4515], 3: [98], 4: [97], 5: [4380], 6: [142], 7: [4350], 8: [203], 9: [4365], 10: [4366], 12: [1748], 13: [1763], 23: [4419], 14: [1762], 15: [4283], 16: [1657], 17: [4298], 18: [4297], 19: [1507], 20: [4388], 21: [1552], 22: [4418], 24: [1464], 25: [4582], 35: [1463], 26: [1373], 27: [4552], 28: [4553], 29: [4554], 30: [1733], 31: [4478], 32: [4493], 33: [4492], 34: [1448], 36: [2745], 37: [3059], 47: [2746], 38: [2805], 39: [3194], 40: [2731], 41: [2730], 42: [3089], 43: [2625], 44: [3149], 45: [2655], 46: [3119], 48: [4532], 49: [473], 59: [564], 50: [4621], 51: [428], 52: [429], 53: [4726], 54: [4727], 55: [534], 56: [4682], 57: [609], 58: [4667], 60: [1755], 61: [3008], 71: [1756], 62: [1605], 63: [3098], 64: [1651], 65: [1652], 66: [1653], 67: [3534], 68: [1727], 69: [3444], 70: [1757], 72: [1244], 73: [5571], 83: [1243], 74: [1394], 75: [1393], 76: [5317], 77: [5332], 78: [1347], 79: [5241], 80: [1212], 81: [1213], 82: [5406], 84: [5312], 8

In [30]:
#set the coupler's interaction to -1 for now - this is to be determined later
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})

In [33]:
hvals={}
for node in list(S.nodes):
    spin0=node
    qubit0=embedding.get(spin0)[0]
    hvals.update({qubit0:0.3})

In [34]:
hvals

{4426: 0.3,
 233: 0.3,
 4515: 0.3,
 98: 0.3,
 97: 0.3,
 4380: 0.3,
 142: 0.3,
 4350: 0.3,
 203: 0.3,
 4365: 0.3,
 4366: 0.3,
 368: 0.3,
 1748: 0.3,
 1763: 0.3,
 1762: 0.3,
 4283: 0.3,
 1657: 0.3,
 4298: 0.3,
 4297: 0.3,
 1507: 0.3,
 4388: 0.3,
 1552: 0.3,
 4418: 0.3,
 4419: 0.3,
 1464: 0.3,
 4582: 0.3,
 1373: 0.3,
 4552: 0.3,
 4553: 0.3,
 4554: 0.3,
 1733: 0.3,
 4478: 0.3,
 4493: 0.3,
 4492: 0.3,
 1448: 0.3,
 1463: 0.3,
 2745: 0.3,
 3059: 0.3,
 2805: 0.3,
 3194: 0.3,
 2731: 0.3,
 2730: 0.3,
 3089: 0.3,
 2625: 0.3,
 3149: 0.3,
 2655: 0.3,
 3119: 0.3,
 2746: 0.3,
 4532: 0.3,
 473: 0.3,
 4621: 0.3,
 428: 0.3,
 429: 0.3,
 4726: 0.3,
 4727: 0.3,
 534: 0.3,
 4682: 0.3,
 609: 0.3,
 4667: 0.3,
 564: 0.3,
 1755: 0.3,
 3008: 0.3,
 1605: 0.3,
 3098: 0.3,
 1651: 0.3,
 1652: 0.3,
 1653: 0.3,
 3534: 0.3,
 1727: 0.3,
 3444: 0.3,
 1757: 0.3,
 1756: 0.3,
 1244: 0.3,
 5571: 0.3,
 1394: 0.3,
 1393: 0.3,
 5317: 0.3,
 5332: 0.3,
 1347: 0.3,
 5241: 0.3,
 1212: 0.3,
 1213: 0.3,
 5406: 0.3,
 1243: 0.3,
 5312:

# anneal

In [29]:

sval = 0.35
pausetime = 10
rev_sch=[[0.0, 1.0], [(1 - sval)/2, sval], [(1 - sval)/2 + pausetime, sval],
        [2*(1 - sval)/2 + pausetime, 1.0]]
estimated_runtime = sampler.solver.estimate_qpu_access_time(num_qubits=len(embedding),
   num_reads=10, anneal_schedule=rev_sch, initial_state = )    
print("Estimate of {:.0f}us on {}".format(estimated_runtime, qpu.solver.name)) 

ValueError: reverse annealing requested (``anneal_schedule`` starts with s==1) but ``initial_state`` not provided

In [7]:
response=sampler.sample_ising(h={},J=couplers,annealing_time=1,num_reads=10)
print(response)

  30 32 33 35 36 39 41 42 45 46 47 48 50 51 52 53 56 ... 5728  energy num_oc.
0 +1 -1 +1 +1 +1 +1 -1 -1 +1 +1 +1 +1 +1 +1 +1 +1 +1 ...   +1 -3520.0       1
1 -1 -1 -1 +1 +1 +1 +1 +1 -1 -1 -1 -1 +1 -1 -1 +1 +1 ...   +1 -3520.0       1
2 +1 -1 +1 +1 +1 +1 -1 -1 +1 -1 -1 -1 +1 -1 -1 +1 +1 ...   +1 -3520.0       1
3 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 -1 -1 -1 -1 ...   -1 -3504.0       1
4 +1 -1 -1 +1 +1 -1 +1 +1 +1 -1 -1 -1 +1 -1 -1 +1 -1 ...   -1 -3500.0       1
5 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 ...   +1 -3500.0       1
6 +1 -1 -1 -1 -1 +1 +1 +1 +1 -1 -1 -1 -1 -1 -1 +1 +1 ...   +1 -3496.0       1
7 +1 +1 -1 +1 +1 -1 +1 +1 +1 -1 -1 -1 +1 -1 -1 +1 +1 ...   -1 -3492.0       1
8 +1 +1 -1 +1 +1 -1 +1 +1 +1 -1 -1 -1 +1 -1 -1 +1 +1 ...   +1 -3476.0       1
9 +1 +1 +1 -1 -1 +1 +1 +1 +1 +1 +1 +1 -1 -1 -1 +1 -1 ...   +1 -3472.0       1
['SPIN', 10 rows, 10 samples, 3600 variables]


In [8]:
dwave.inspector.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=abcf2f1e-93b1-4e7c-9d95-4b099d0d8427

'http://127.0.0.1:18000/?problemId=abcf2f1e-93b1-4e7c-9d95-4b099d0d8427'